# Kickstarter

In [ ]:
import datatable as dt
import mmcv
import os
import torch
import shutil

from datatable import f
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from tqdm.auto import tqdm
from utilpy import sv, ld

dt.init_styles()

wdir = '/home/yu/OneDrive/Construal'
model_dir = f'{wdir}/pretrained_models/mmdetection'
data_dir = f'{wdir}/data'

os.chdir(wdir)

## config Model

In [8]:
# Specify the path to model config and checkpoint file
config_file = f'{model_dir}/configs/lvis/mask_rcnn_x101_64x4d_fpn_sample1e-3_mstrain_1x_lvis_v1.py'
# config_file = f'{model_dir}/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'

checkpoint_file = f'{model_dir}/checkpoints/mask_rcnn_x101_64x4d_fpn_sample1e-3_mstrain_1x_lvis_v1-43d9edfe.pth'
# checkpoint_file = f'{model_dir}/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

## unit test

> plot results and save to disk

In [ ]:
# get pid of "product design"
# pjson = ld('pjson', folder=data_dir)
# pids = pjson[(f.category=='Product Design'),f.pid].to_list()[0]

# print(f'{len(pids)=}')

# init models
device = 'cuda:0'
model = init_detector(config_file, checkpoint_file, device=device)

# get pids
img_dir = f'{data_dir}/sharing/object-detect-example-v3'
imgs = os.listdir(img_dir)

for img_path in ['8.png']:
    out_img_path = f'{img_dir}/out-{img_path}'
    img_path = f'{img_dir}/{img_path}'

    # only process png
    if (not img_path.endswith('.png')) or (not os.path.exists(img_path)):
        continue

    # detect
    res = inference_detector(model, img_path)

    # print visualization
    show_result_pyplot(model, img_path, res, palette=None, score_thr=0.5)

    # save results to disk
    model.show_result(img_path, res, 
                      score_thr=0.5,
                      out_file=f'{out_img_path}', show=False)

## deploy

In [6]:
# get pid of "product design"
pjson = ld('pjson', folder=data_dir)
pids = pjson[(f.category=='Product Design') | (f.category=='Accessories'),f.pid].to_list()[0]

print(f'{len(pids)=}')

"pjson.feather" (167.5 MB) loaded (<1s) (2023-05-14 7:20 PM)
len(pids)=3759


In [ ]:
# Start Detecting!
def obj_detect(pid, model):
    pdir = f'{data_dir}/kickstarter-image/{pid}'
    output = {}
    if not os.path.exists(pdir):
        return

    for i, file in enumerate(os.listdir(pdir)):
        if file.endswith('.jpg'):
            img = f'{pdir}/{file}'
            res = inference_detector(model, img)
            
            # get cat_idx and cat_n
            # res[0]: box
            # res[1]: segment
            box_res = {}
            for cat_idx, cat in enumerate(res[0]):
                if len(cat)>0:
                    box_res[cat_idx] = {'n':len(cat),
                                        'prob':cat[:,-1].tolist()}
            
            output[file] = box_res

    # save results
    save_dir = f'{pdir}/object results'
    save_path = f'{save_dir}/mrcnn_lvis.pt'
    if not os.path.exists(save_path):
        os.mkdir(save_dir)
    torch.save(output, save_path)
    
# device = f'cuda:0'
device = 'cpu'
model = init_detector(config_file, checkpoint_file, device=device)

for pid in tqdm(pids[99:100]):
    obj_detect(pid, model)

## retreving results for sharing


In [2]:
# get pid of "product design"
ld('pjson', path=data_dir)
pids = pjson[(f.category=='Product Design') | (f.category=='Accessories'),f.pid].to_list()[0]
print(f'{len(pids)=}')

"pjson.feather" (167.5 MB) loaded (<1s) (2021-10-18 4:03 PM)
len(pids)=3759


In [28]:
# ---------------------
#  copy obj-detect results from the original folder to the sharing folder
# ---------------------

# create a new folder in the sharing folder
target = f'{data_dir}/sharing/object-detect-results'

if os.path.exists(target):
    shutil.rmtree(target)
os.mkdir(target)

def save_sharing(pid):
    pdir = f'{data_dir}/kickstarter-image/{pid}/object results'
    if not os.path.exists(pdir):
        return

    target_dir = f'{target}/{pid}'
    if not os.path.exists(target_dir):
        shutil.copytree(pdir, target_dir)

for pid in tqdm(pids):
    save_sharing(pid)

100%|██████████| 3759/3759 [00:00<00:00, 8545.16it/s]


In [2]:
# collect results into a single dataset "df_objdet"
wdir = '/home/yu/OneDrive/Construal'
os.chdir(wdir)

import torch

root_dir = '/home/yu/OneDrive/Construal/data/sharing/object-detect-results'

detect_res = {}
for i, pid in enumerate(os.listdir(root_dir)):
    res = torch.load(f'{root_dir}/{pid}/mrcnn_lvis.pt')
    detect_res[pid] = res
    

detect_df = []
for pid, res in detect_res.items():
    for jpg, labels in res.items():
        for label_id, label_counts in labels.items():
            for inst_id, prob in enumerate(label_counts['prob']):
                detect_df.append((pid, jpg, label_id, inst_id, prob))
detect_df = dt.Frame(detect_df, names=['pid', 'jpg', 'label_id', 'inst_id', 'prob'])

sv('df_objdet')

"df_objdet" saved as "df_objdet.feather" (82.1 MB) (1s) (2021-10-18 4:18 PM)


In [1]:
# (required) add label_name to df_objdet (R)
wdir = '/home/yu/OneDrive/Construal'
setwd(wdir)

ld(detect_df)
ld(lvis_dist)

detect_df = lvis_dist[, .(label_id=id, label_name=name)
    ][detect_df, on=.(label_id)]

npid = detect_df[, uniqueN(pid)]
cat('N unique pids:', npid, '\n')

detect_df[1:2]

sv(detect_df)

"df_objdet.feather" (82.1 MB) loaded (0.69 secs) (2021-10-18 4:18 PM)
"lvis_dist.feather" (80.4 KB) loaded (0 secs) (2021-10-18 4:18 PM)
N unique pids: 3750 


label_id,label_name,pid,jpg,inst_id,prob
<int>,<chr>,<chr>,<chr>,<int>,<dbl>
11,antenna,563540633,profile_full.jpg,0,0.11811149
11,antenna,563540633,profile_full.jpg,1,0.02381403


"df_objdet" saved as "df_objdet.feather" (112.2 MB) (0.64 secs, 2021-10-18 16:18:19)


# Kickstarter (faces)

In [1]:
import concurrent
import datatable as dt
import torch
import shutil

from deepface import DeepFace
from datatable import f
from tqdm.auto import tqdm
from retinaface import RetinaFace

dt.init_styles()

wdir = '/home/yu/OneDrive/Construal'
model_dir = f'{wdir}/pretrained_models/mmdetection'
data_dir = f'{wdir}/data'

os.chdir(wdir)

In [ ]:
# get pid of "product design"
ld('pjson', path=data_dir)
pids = pjson[(f.category=='Product Design') | (f.category=='Accessories'),f.pid].to_list()[0]

print(f'{len(pids)=}')

# Start Detecting!
def face_detect(pid):
    img = f'{data_dir}/kickstarter-image/{pid}/profile_full.jpg'
    if not os.path.exists(img):
        return

    faces = RetinaFace.extract_faces(img)
    n_faces = len(faces)
    n_happy_faces = 0

    if n_faces > 0:
        for face in faces:
            obj = DeepFace.analyze(face, detector_backend='skip', actions=['emotion'])
            if obj['dominant_emotion'] == 'happy':
                n_happy_faces += 1
    
    return pid, n_faces, n_happy_faces
            
            
# out_faces = []
# for pid in tqdm(pids[:10]):
#     out_faces.append(face_detect(pid))

with concurrent.futures.ProcessPoolExecutor() as executor:
    out_faces = list(executor.map(face_detect, pids))

out_faces = [f for f in out_faces if f is not None]  # remove None
out_faces = dt.Frame(out_faces, names=['pid', 'n_faces', 'n_happy_faces'])

sv('out_faces')

# Fiver

Compatible with mmdetect>=3.0, no longer work with mmdetect2.x!

In [57]:
import os
import pandas as pd
import torch

from pyarrow.feather import write_feather, read_feather
from mmdet.apis import init_detector, inference_detector
from tqdm.auto import tqdm
from pathlib import Path

wdir = Path('/home/yu/OneDrive/Construal')
data_dir = Path('/home/yu/chaoyang/research-resources/fiver-raw-from-amrita/Image Folder')

model_dir = f'{wdir}/pretrained_models/mmdetection'
os.chdir(wdir)

In [ ]:
from mmdet.apis import init_detector, inference_detector

config_file = 'pretrained_models/lvis/mask-rcnn_x101-64x4d_fpn_sample1e-3_ms-1x_lvis-v1.py'
checkpoint_file = 'pretrained_models/lvis/mask_rcnn_x101_64x4d_fpn_sample1e-3_mstrain_1x_lvis_v1-43d9edfe.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')  # or device='cuda:0'
inference_detector(model, 'pretrained_models/test/demo.jpg')

### config Model

In [14]:
# Specify the path to model config and checkpoint file
config_file = 'pretrained_models/lvis/mask-rcnn_x101-64x4d_fpn_sample1e-3_ms-1x_lvis-v1.py'

checkpoint_file = 'pretrained_models/lvis/mask_rcnn_x101_64x4d_fpn_sample1e-3_mstrain_1x_lvis_v1-43d9edfe.pth'

device = 'cuda:0'
model = init_detector(config_file, checkpoint_file, device=device)

Loads checkpoint by local backend from path: pretrained_models/lvis/mask_rcnn_x101_64x4d_fpn_sample1e-3_mstrain_1x_lvis_v1-43d9edfe.pth


### run detection

In [31]:
# function to detect objects
def obj_detect(pid, model):
    # pdir: project folder
    pdir = data_dir/pid

    if not pdir.exists():
        return

    for i, img_name in enumerate(os.listdir(pdir)):
        if img_name.endswith('_main.jpg'):
            img = f'{pdir}/{img_name}'
            try:
                res = inference_detector(model, img)
            except:
                return
            
            # collect cat_idx and prob
            output = {}  # {file: {cat_idx: {n: , prob: }}}

            cat_idxes = res.pred_instances.to_dict()['labels'].tolist()
            probs = res.pred_instances.to_dict()['scores'].tolist()

            for cat_idx, prob in zip(cat_idxes, probs):
                if cat_idx not in output:
                    output[cat_idx] = {'n': 1, 'prob': [prob]}
                else:
                    output[cat_idx]['n'] += 1
                    output[cat_idx]['prob'].append(prob)

    # create folder to save results
    save_dir = wdir/'data/fiver-object-detect-results/'
    save_dir.mkdir(exist_ok=True)

    # save results
    save_path = save_dir/f'{pid}_objects.pt'
    if not save_path.exists():
        torch.save(output, save_path)
    
# get pids
pids = []
for (dirpath, dirnames, filenames) in os.walk(data_dir):
    for filename in filenames:
        if filename.endswith('_main.jpg'):
            pids.append(filename.split('_')[0])    

# Start Detecting!
for pid in tqdm(pids):
    obj_detect(pid, model)

 16%|█▌        | 665/4268 [04:32<21:14,  2.83it/s]Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7f7d57e35d80>
Traceback (most recent call last):
  File "/usr/lib/python3.10/tempfile.py", line 589, in __del__
    self.close()
  File "/usr/lib/python3.10/tempfile.py", line 585, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpshgcf0zy/tmpf9njhp2_.py'
Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7f7d57e35d80>
Traceback (most recent call last):
  File "/usr/lib/python3.10/tempfile.py", line 589, in __del__
    self.close()
  File "/usr/lib/python3.10/tempfile.py", line 585, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpdxbiigu2/tmpwsj0jf8f.py'
Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7f7d57e35d80>
Traceback (most recent call last):
  File "/usr/lib/python3.10/tempfile.py", line 589, in __del__
    self.close()
  File "/usr/lib/py

### merge results

In [63]:
# collect results into a dict
res_dir = wdir/'data/fiver/object-detect-results'

detect_res = {}
for i, path in enumerate(os.listdir(res_dir)):
    pid = path.split('_')[0]
    res = torch.load(res_dir/path)
    detect_res[pid] = res
    

# convert to dataframe
detect_df = []
for pid, res in detect_res.items():
    for label_id, label_counts in res.items():
        for inst_id, prob in enumerate(label_counts['prob']):
            detect_df.append((pid, label_id, inst_id, prob))
detect_df = pd.DataFrame(detect_df, columns=['pid', 'label_id', 'inst_id', 'prob'])

# add "label_name"
label_name = read_feather('data/lvis_dist.feather')[['id', 'name']]
detect_df = detect_df.merge(label_name, left_on='label_id', right_on='id') \
    .drop(columns=['id']) \
    .rename({'name': 'label_name'}, axis=1)

write_feather(detect_df, 'data/fiver/object-detect-res.feather')